In [0]:
#Mount the google drive
from google.colab import drive
drive.mount( '/content/gdrive', force_remount=True )


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Import necessary libraries
import os
import sys
from time import time
from PIL import Image
import numpy as np
from glob import glob
from tqdm import tqdm

from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
#update the directory and move the process to required location
sys.path.append("/content/gdrive/My Drive/TSAI_EVA4/Session 14-15A/")
os.chdir('/content/gdrive/My Drive/TSAI_EVA4/Session 14-15A/')  


In [0]:
#Initialize path for folders of Background, Foreground, FG_BG, FG_BG_MASK images
path = os.getcwd()
bg_path = path + '/Background'
bg_img_resize = (224,224)
bg_out_path  = path + '/Baskball_Players_DepthEstimation/BG/' 

fg_path = path + '/Foreground'
fg_img_resize = (100,100)
fg_out_path  = path + '/Baskball_Players_DepthEstimation/FG/' 

fg_mask_path = path + '/Mask'
fg_mask_img_resize = (100,100)
fg_mask_out_path  = path + '/Baskball_Players_DepthEstimation/FG_MASK/' 



In [0]:
#Read background images. Resize to square images and redue the file size to avoid memory issue 

from time import time
bg_list=[]
st = time()
for bidx, b in enumerate(os.listdir(bg_path)):
  img = Image.open(os.path.join(bg_path, b))
  img = img.resize(bg_img_resize, Image.ANTIALIAS)
  img.save(os.path.join(bg_out_path, 'bg_{0:03d}'.format(bidx)), "JPEG", optimize=True,quality=30)
  bg_list.append(img)
  del img,b,bidx
print("time taken", (time()-st)/60)
print(len(bg_list))

time taken 1.9267815589904784
100


In [0]:
#Read Foreground images. Resize to square images and redue the file size  
st = time()
for fidx, f in enumerate(os.listdir(fg_path)):
  img = Image.open(os.path.join(fg_path, f))
  img = img.resize(fg_img_resize, Image.ANTIALIAS)
  img.save(os.path.join(fg_out_path, 'fg_{0:03d}'.format(fidx)), "PNG", optimize=True,quality=30)
  del img,f,fidx
print("time taken",(time()-st)/60)

time taken 1.1867639183998109


In [0]:
#Read Foreground Mask images. Resize to square images and redue the file size
#Generate Black and white single channel masks for foreground images
def generate_bw_mask(image):
    #make mask of where the transparent bits are
    img_arr = np.array(image)
    img_arr[:, :, :-1][img_arr[:, :, :-1] == 0] = 255
    img_arr[img_arr > 0] = 255
    del image
    return Image.fromarray(img_arr)

st = time()
fg_list, fg_mask_list=[], []
for fidx, f in enumerate(os.listdir(fg_mask_path)):
  img = Image.open(os.path.join(fg_mask_path, f))
  img = img.resize(fg_img_resize, Image.ANTIALIAS)
  img.save(os.path.join(fg_mask_out_path, 'fg_mask_{0:03d}'.format(fidx)), "PNG", optimize=True,quality=30)
  fg_list.append(img)
  fg_mask_list.append(generate_bw_mask(img))

  fg_list.append(img.transpose(Image.FLIP_LEFT_RIGHT))
  fg_mask_list.append(generate_bw_mask(img.transpose(Image.FLIP_LEFT_RIGHT)))

  del img,f,fidx
print("time taken",(time()-st)/60)
print(len(fg_list), len(fg_mask_list))

time taken 0.1539769212404887
200 200


In [0]:
#Generate Random co ordinate positions to overlay foreground on background images. To keep consistent of postitions, the co-ords are made static
# place=[]
# import random 
# for i in range(20):
#     num1 = random.randint(10, 150)
#     num2 = random.randint(10,150)
#     n=tuple([num1,num2])
#     place.append(n)

co_ord = [(35, 94), (107, 74), (121, 29), (103, 79), (91, 45), (87, 127), (83, 36), (101, 38), (27, 126), (99, 47), (90, 45), (112, 140), (28, 99), (38, 144), (98, 127), (143, 79), (61, 75), (88, 103), (76, 11), (145, 63)]


In [0]:
#Create overlayed images of FG-BG and FG-BG Masks.

def overlay_transparent(bg, fg, x,y):
    overlayed_bg = bg.copy()
    overlayed_bg.paste(fg, (x, y), fg)
    return overlayed_bg

#Initialize output folder paths to save the outputs
fg_bg_path = path + '/Baskball_Players_DepthEstimation/FG_BG/' 
mask_fg_bg_path = path + '/Baskball_Players_DepthEstimation/FG_BG_MASK/' 

st=time()

for bidx, bg_img in enumerate(tqdm(bg_list[14:100])):   
    fgi=0
    for fg_img in fg_list:
        fgi+=1
        pos=0
        for p in co_ord:   
            pos+=1
            b_copy = bg_img.copy()
            org_fg_bg = overlay_transparent(b_copy,fg_img,p[0],p[1])
            org_fg_bg.save(os.path.join(fg_bg_path, "bg_{0:03d}_fg_{1:03d}_P_{2:03d}.jpeg".format(bidx, fgi, pos)))
    del fg_img, b_copy, org_fg_bg, fgi, p, pos

    
    fgm=0
    for fg_mask in fg_mask_list:
        fgm+=1
        pos=0
        for p in co_ord:   
            pos+=1
            bgw = Image.fromarray(np.zeros_like(bg_img))
            bgw_copy = bgw.copy()
            fg_mask_copy = Image.fromarray(fg_mask)
            bgw_copy.paste(fg_mask_copy, (p[0], p[1]), fg_mask_copy)
            bgw_copy= bgw_copy.convert(mode='L') 
            
            bgw_copy.save(os.path.join(mask_fg_bg_path, 'bg_{0:03d}_fg_{1:03d}_P_{2:03d}_mask.jpeg'.format(bidx, fgm, pos)))
            
    del bg_img, bgw,bgw_copy, fg_mask, fg_mask_copy, pos, fgm, bidx 










  0%|          | 0/86 [00:00<?, ?it/s]








  1%|          | 1/86 [01:20<1:54:31, 80.84s/it]








  2%|▏         | 2/86 [02:43<1:53:52, 81.33s/it]








  3%|▎         | 3/86 [04:09<1:54:27, 82.74s/it]








  5%|▍         | 4/86 [05:30<1:52:36, 82.39s/it]








  6%|▌         | 5/86 [06:52<1:50:51, 82.12s/it]








  7%|▋         | 6/86 [08:12<1:48:42, 81.53s/it]








  8%|▊         | 7/86 [09:32<1:46:47, 81.10s/it]








  9%|▉         | 8/86 [10:52<1:45:00, 80.77s/it]








 10%|█         | 9/86 [12:11<1:42:47, 80.09s/it]








 12%|█▏        | 10/86 [13:29<1:40:53, 79.65s/it]








 13%|█▎        | 11/86 [14:49<1:39:43, 79.78s/it]








 14%|█▍        | 12/86 [16:12<1:39:26, 80.63s/it]








 15%|█▌        | 13/86 [18:03<1:49:15, 89.80s/it]








 16%|█▋        | 14/86 [20:09<2:00:53, 100.74s/it]








 17%|█▋        | 15/86 [22:20<2:09:40, 109.59s/it]








 19%|█▊        | 16/86 [24:34<2:16:34, 117.07s/it]








 20%|█▉        | 17/86

KeyboardInterrupt: ignored

In [0]:
# Create depth images using DenseDpeth model

!cd DenseDepth; python test.py